*Last Updated: 24.07.27*
<br><br>네이버 로그인 후 본부원과 봉사자 개개인의 네이버 폼 이미지 링크에 접속하여 서명 이미지를 자동으로 다운로드 받습니다(파일명: `전화번호.jpeg`). 
<br>그 후 모든 본부원과 봉사자 정보가 담긴 수기명부에 서명 이미지 자동으로 삽입합니다.
<br>※데이터셋의 모든 인물 정보는 허구입니다.

In [ ]:
#-*-coding:utf-8-*-

In [10]:
import xlsxwriter
import pandas as pd
import openpyxl
import shutil
import os
import time
import pyperclip
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains, Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from PIL import Image

In [32]:
root_path = "C:/Users/2277007/Desktop/IHCO/dataset/"
img_root_path = "C:/Users/2277007/Desktop/IHCO/img/" # 서명 이미지 저장 폴더 
src = root_path + "설문조사_배포용.xlsx"
dst = root_path + "봉사자_전자서명.xlsx"
src_df = pd.read_excel(src, engine="openpyxl")

In [8]:
wb = openpyxl.load_workbook(src)
ws = wb.get_sheet_by_name('설문 답변')
print(ws.cell(row=2, column=17).hyperlink.target)
links = []
for i in range(2, ws.max_row + 1):  # 2nd arg in range() not inclusive, so add 1
    links.append(ws.cell(row=i, column=17).hyperlink.target) ## sample dataset에서는 인식되지 않습니다.
src_df['link'] = links

https://survey.naver.com/form/imageView?src=https%3A%2F%2Fsurvey.naver.com%2Fform%2Fimages%example.jpeg
C:\Users\2277007\AppData\Local\Temp\ipykernel_17912\426241985.py:2: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
ws = wb.get_sheet_by_name('설문 답변')


In [35]:
src_df = src_df.loc[:, ['1. 이름 (예시:김보건)(*)', '5. 연락처(*)', '6. 1365 아이디(*)', '7. VMS 아이디(*)', 'link']]
src_df.rename(columns = {'1. 이름 (예시:김보건)(*)': '성함', '5. 연락처(*)' : '전화번호', '6. 1365 아이디(*)': '1365 ID', '7. VMS 아이디(*)': 'VMS ID'}, inplace = True)
src_df.head()

,성함,전화번호,1365 ID,VMS ID,link
0,김철수,010-1234-5678,example1,example1,https://survey.naver.com/form/imageView?src=ht...
1,박영희,010-1111-1111,example2,example10,https://survey.naver.com/form/imageView?src=ht...
2,김민준,010-4444-4444,example3,example3,https://survey.naver.com/form/imageView?src=ht...
3,이서윤,010-3333-3333,example4,example4,https://survey.naver.com/form/imageView?src=ht...
4,송이겸,010-0000-1111,example5,example5,https://survey.naver.com/form/imageView?src=ht...


In [10]:
class NaverLoginService():

    def __init__(self, phone_list, link_list):
        self.driver = None
        self.phone_list = phone_list
        self.link_list = link_list

    def open_web_mode(self):
        self.driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
        self.driver.set_page_load_timeout(10)

    def close_browser(self):
        if self.driver:
            self.driver.quit()
            self.driver = None

    def login(self):
        self.driver.get("https://nid.naver.com/nidlogin.login")
        time.sleep(2)  # 페이지 로딩 대기

        test_id = "ID" # 네이버 아이디 입력
        test_passwd = "PW" # 네이버 비밀번호 입력 

        # 네이버 아이디 입력
        id_input = self.driver.find_element(By.ID, "id")
        id_input.click()
        pyperclip.copy(test_id)
        actions = ActionChains(self.driver)
        actions.key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
        time.sleep(1)  # 입력 후 잠시 대기

        # 네이버 패스워드 입력
        pw_input = self.driver.find_element(By.ID, "pw")
        pw_input.click()
        pyperclip.copy(test_passwd)
        actions = ActionChains(self.driver)
        actions.key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
        time.sleep(1)  # 입력 후 잠시 대기

        # 로그인 버튼 클릭
        self.driver.find_element(By.ID, "log.login").click()

        # 로그인 후 '새로운 환경' 알림에서 '등록안함' 버튼 클릭
        try:
            element = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "span.btn_cancel"))
            )
            element.click()
        except:
            print("기기 등록 '등록안함' 버튼을 찾을 수 없습니다.")
    
    def get_image(self):
        user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
        for p, url in zip(self.phone_list, self.link_list):
            self.driver.get(url)
            image = self.driver.find_element(By.XPATH, '/html/body/div/div[2]/div/img').get_attribute("src")
            #print(image)
            self.driver.get(image)
            time.sleep(5)
            down_path = "C:/Users/2277007/Downloads"
            new_file_name = max([down_path + "\\" + f for f in os.listdir(down_path)],key=os.path.getctime)
            shutil.move(new_file_name,os.path.join(img_root_path + p + ".jpeg"))

naver_service = NaverLoginService(src_df['전화번호'].tolist(), src_df['link'].tolist())
naver_service.open_web_mode()
naver_service.login()
time.sleep(5)
naver_service.get_image()
naver_service.close_browser()

In [ ]:
workbook = xlsxwriter.Workbook(dst)
worksheet = workbook.add_worksheet()
cell_width = 60.0
cell_height = 50.0

for i, (p, n, o, v) in enumerate(zip(src_df['전화번호'].tolist(), src_df['성함'].tolist(), src_df['1365 ID'].tolist(), src_df['VMS ID'].tolist())):
    try:
        im = Image.open(img_root_path+str(p)+'.jpeg')
    except: print("Error:", p,n)
    image_width, image_height = im.size
    if image_width < image_height: print("size warning:", p, n) # 서명 이미지가 세로일 때 경고 출력 
    scale = cell_width/image_width
    worksheet.set_column('A:E', cell_width)
    worksheet.set_row(i+1, image_height*scale)
    worksheet.write(f"A{i+1}", n) # 성함
    worksheet.write(f"B{i+1}", p) # 전화번호
    worksheet.write(f"C{i+1}", o) # 1365 ID
    worksheet.write(f"D{i+1}", v) # VMS ID
    worksheet.insert_image(f'E{i+1}', img_root_path+p+'.jpeg',
                       {'x_scale': scale, 'y_scale': scale})
workbook.close()